In [1]:
!pip install torchtext==0.6.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.3 MB/s eta 0:00:00
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.15.1
    Uninstalling torchtext-0.15.1:
      Successfully uninstalled torchtext-0.15.1


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import torch
from torchtext.data import Field, LabelField, BucketIterator, TabularDataset
import spacy
import re

In [2]:
import drive.MyDrive.NLP_stuff.Sentiment_analysis.dataset_loader_torchtext as dlt
dataset_loader = dlt.CustomDatasetLoader('/content/drive/MyDrive/NLP_stuff/Sentiment_analysis/train.csv')

In [3]:
train_data, test_data = dataset_loader.get_data()
TEXT, LABEL = dataset_loader.get_fields()

In [4]:
print(len(train_data))

83005


In [6]:
import torch
from torchtext import data

BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, test_iterator = dataset_loader.get_iterators(batch_size=BATCH_SIZE)

LSTM

In [8]:
import torch.nn as nn

class RNN_model(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * (1 + bidirectional), output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):        
        embedded = self.dropout(self.embedding(text))
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.to('cpu'))
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        return self.fc(hidden)

GRU

In [7]:
import torch.nn as nn

class RNN_model(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.rnn = nn.GRU(embedding_dim, 
                          hidden_dim, 
                          num_layers=n_layers, 
                          bidirectional=bidirectional, 
                          dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * (1 + bidirectional), output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):        
        embedded = self.dropout(self.embedding(text))
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.to('cpu'))
        packed_output, hidden = self.rnn(packed_embedded)
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        return self.fc(hidden)


Transformer

In [136]:
class AttentionSentimentClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, nhead, num_layers, num_classes):
        super(AttentionSentimentClassifier, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.attention = nn.MultiheadAttention(embedding_dim, nhead)
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(embedding_dim, nhead),
            num_layers=num_layers)
        self.fc = nn.Linear(embedding_dim, num_classes)

    def forward(self, x, x_len):
        embedded = self.embedding(x)
        embedded = embedded.permute(1, 0, 2)
        query = key = value = embedded
        attention_output, _ = self.attention(query, key, value)
        attention_output = attention_output.permute(1, 0, 2)
        encoded = self.transformer_encoder(attention_output)
        encoded = encoded.permute(1, 0, 2)
        logits = self.fc(encoded[:, 0])
        return logits

In [137]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
nhead = 5
OUTPUT_DIM = 3
N_LAYERS = 4
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = AttentionSentimentClassifier(INPUT_DIM, EMBEDDING_DIM, nhead,N_LAYERS, OUTPUT_DIM  )

In [20]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 290
OUTPUT_DIM = 3
N_LAYERS = 3
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN_model(INPUT_DIM, 
                  EMBEDDING_DIM, 
                  HIDDEN_DIM, 
                  OUTPUT_DIM, 
                  N_LAYERS, 
                  BIDIRECTIONAL, 
                  DROPOUT, 
                  PAD_IDX)

In [21]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 4,914,283 trainable parameters


In [22]:
pretrained_embeddings = TEXT.vocab.vectors
print(pretrained_embeddings.shape)

torch.Size([11959, 100])


In [23]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1602, -0.2917, -1.6108,  ...,  0.1727,  0.4389,  0.8895],
        [ 0.2303, -1.4974,  0.4848,  ...,  0.5191, -0.7186, -1.1488],
        [-0.5067,  0.5431,  0.3948,  ..., -0.8985,  1.5412,  0.1602],
        ...,
        [-0.7911,  0.8509, -0.8855,  ..., -0.4691,  0.3817, -1.4397],
        [-0.6763,  0.1671,  0.9469,  ..., -0.3254, -0.2345,  0.2310],
        [ 0.0875,  0.6363, -1.1615,  ...,  0.8988,  2.3986, -0.0352]])

In [24]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)
print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.5067,  0.5431,  0.3948,  ..., -0.8985,  1.5412,  0.1602],
        ...,
        [-0.7911,  0.8509, -0.8855,  ..., -0.4691,  0.3817, -1.4397],
        [-0.6763,  0.1671,  0.9469,  ..., -0.3254, -0.2345,  0.2310],
        [ 0.0875,  0.6363, -1.1615,  ...,  0.8988,  2.3986, -0.0352]])


In [25]:
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

optimizer = optim.Adam(model.parameters())
scheduler = lr_scheduler.LinearLR(optimizer, start_factor=1, end_factor=0.001, total_iters=6)
criterion = nn.CrossEntropyLoss().to(device)

model = model.to(device)

In [26]:
def categorical_accuracy(preds, y):
    top_pred = preds.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [27]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in tqdm(iterator):
        
        optimizer.zero_grad()
        
        
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)

        loss = criterion(predictions, batch.label)
        
        acc = categorical_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
    scheduler.step()
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [28]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = categorical_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [29]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [30]:
from tqdm import tqdm

In [31]:
N_EPOCHS = 20

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')
    print('\n')

100%|██████████| 1297/1297 [00:17<00:00, 73.37it/s]


Epoch: 01 | Epoch Time: 0m 18s
	Train Loss: 0.864 | Train Acc: 61.69%
	 Val. Loss: 0.749 |  Val. Acc: 68.18%




100%|██████████| 1297/1297 [00:18<00:00, 71.78it/s]


Epoch: 02 | Epoch Time: 0m 18s
	Train Loss: 0.760 | Train Acc: 67.41%
	 Val. Loss: 0.703 |  Val. Acc: 70.43%




100%|██████████| 1297/1297 [00:17<00:00, 73.29it/s]


...

Epoch: 18 | Epoch Time: 0m 18s
	Train Loss: 0.623 | Train Acc: 74.04%
	 Val. Loss: 0.656 |  Val. Acc: 73.19%




100%|██████████| 1297/1297 [00:18<00:00, 68.79it/s]


Epoch: 19 | Epoch Time: 0m 19s
	Train Loss: 0.625 | Train Acc: 73.78%
	 Val. Loss: 0.656 |  Val. Acc: 73.14%




100%|██████████| 1297/1297 [00:17<00:00, 73.04it/s]


Epoch: 20 | Epoch Time: 0m 18s
	Train Loss: 0.624 | Train Acc: 73.76%
	 Val. Loss: 0.656 |  Val. Acc: 73.13%




In [32]:
model.load_state_dict(torch.load('model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.655 | Test Acc: 73.06%


In [33]:
def predict_class(model, sentence):
    model.eval()
    tokenized = dlt.tokenize_english(sentence)
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    preds = model(tensor, length_tensor)
    max_preds = preds.argmax(dim = 1)
    return max_preds.item()

In [34]:
sentence="Penalty on sleeping in RM is removed on behalf of Shubham Kumar"
pred_class = predict_class(model, sentence)
print(LABEL.vocab.itos[pred_class])

-1


In [35]:
import csv
import chardet

FILENAME = '/content/drive/MyDrive/NLP_stuff/Sentiment_analysis/test.csv'
with open(FILENAME, 'rb') as f:
    result = chardet.detect(f.read())

with open(FILENAME, newline='', encoding=result['encoding']) as csvfile:
    csv_reader = csv.reader(csvfile, delimiter=',', quotechar='"')

    # Skip the header row
    next(csv_reader)

    f_open = open("answer1.txt","w")
    for row in csv_reader:
        pred = LABEL.vocab.itos[predict_class(model, str(row[1]))]
        s= str(pred)
        s+='\n'
        f_open.write(s)
    f_open.close()